## Bulding the image

Clone the course repo if you haven't:

```
git clone https://github.com/alexeygrigorev/mlbookcamp-code.git
```

Go to the `course-zoomcamp/cohorts/2022/05-deployment/homework` folder and 
execute the following:


```bash
docker build -t zoomcamp-model:v001 .
```

> **Note:** If you have troubles building the image, you can 
> use the image we built and published to docker hub:
> `svizor42/zoomcamp-model:v001`


## Question 1

Run it to test that it's working locally:

```bash
docker run -it --rm -p 9696:9696 zoomcamp-model:v001
```

And in another terminal, execute `q6_test.py` file:

```bash
python q6_test.py
```

You should see this:

```
{'get_card': True, 'get_card_probability': <value>}
```

Here `<value>` is the probability of getting a credit card. You need to choose the right one.

* 0.289
* 0.502
* 0.769 [X]
* 0.972

Now you can stop the container running in Docker.

In [6]:
import requests


url = "http://localhost:9696/predict"

client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
response = requests.post(url, json=client).json()

print(response)

{'get_card': True, 'get_card_probability': 0.7692649226628628}


## Installing `kubectl` and `kind`

You need to install:

* `kubectl` - https://kubernetes.io/docs/tasks/tools/ (you might already have it - check before installing)
* `kind` - https://kind.sigs.k8s.io/docs/user/quick-start/

In [36]:
!kind

kind creates and manages local Kubernetes clusters using Docker container 'nodes'

Usage:
  kind [command]

Available Commands:
  build       Build one of [node-image]
  completion  Output shell completion code for the specified shell (bash, zsh or fish)
  create      Creates one of [cluster]
  delete      Deletes one of [cluster]
  export      Exports one of [kubeconfig, logs]
  get         Gets one of [clusters, nodes, kubeconfig]
  help        Help about any command
  load        Loads images into nodes
  version     Prints the kind CLI version

Flags:
  -h, --help              help for kind
      --loglevel string   DEPRECATED: see -v instead
  -q, --quiet             silence all stderr output
  -v, --verbosity int32   info log verbosity, higher value produces more output
      --version           version for kind

Use "kind [command] --help" for more information about a command.


## Question 2

What's the version of `kind` that you have? 

Use `kind --version` to find out.

In [37]:
!kind --version

kind version 0.17.0


## Creating a cluster

Now let's create a cluster with `kind`:

```bash
kind create cluster
```

And check with `kubectl` that it was successfully created:

```bash
kubectl cluster-info
```


In [38]:
!kind create cluster

Creating cluster "kind" ...
 â€¢ Ensuring node image (kindest/node:v1.25.3) ðŸ–¼  ...
 âœ“ Ensuring node image (kindest/node:v1.25.3) ðŸ–¼
 â€¢ Preparing nodes ðŸ“¦   ...
 âœ“ Preparing nodes ðŸ“¦ 
 â€¢ Writing configuration ðŸ“œ  ...
 âœ“ Writing configuration ðŸ“œ
 â€¢ Starting control-plane ðŸ•¹ï¸�  ...
 âœ“ Starting control-plane ðŸ•¹ï¸�
 â€¢ Installing CNI ðŸ”Œ  ...
 âœ“ Installing CNI ðŸ”Œ
 â€¢ Installing StorageClass ðŸ’¾  ...
 âœ“ Installing StorageClass ðŸ’¾
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Have a nice day! ðŸ‘‹


In [39]:
!kubectl cluster-info

Kubernetes control plane is running at https://127.0.0.1:50638
CoreDNS is running at https://127.0.0.1:50638/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


## Question 3

What's the smallest deployable computing unit that we can create and manage 
in Kubernetes (`kind` in our case)?

* Node
* Pod [X] ?
* Deployment
* Service

## Question 4

Now let's test if everything works. Use `kubectl` to get the list of running services.

What's the `Type` of the service that is already running there?

* ClusterIP [X]
* NodePort
* LoadBalancer
* ExternalName

In [40]:
!kubectl get services

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   42s


## Question 5

To be able to use the docker image we previously created (`zoomcamp-model:v001`),
we need to register it with `kind`.

What's the command we need to run for that?

* `kind create cluster`
* `kind build node-image`
* `kind load docker-image` [X]
* `kubectl apply`


In [41]:
!kind load docker-image zoomcamp-model:v001

Image: "" with ID "sha256:52a64ef0493c166b14d35d4e02b51149fcde2c76a5ea4c81f6f007944c1dcf44" not yet present on node "kind-control-plane", loading...


## Question 6

Now let's create a deployment config (e.g. `deployment.yaml`):

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: credit-card
spec:
  selector:
    matchLabels:
      app: credit-card
  replicas: 1
  template:
    metadata:
      labels:
        app: credit-card
    spec:
      containers:
      - name: credit-card
        image: <Image>
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: <Memory>
            cpu: <CPU>
        ports:
        - containerPort: <Port>
```

Replace `<Image>`, `<Memory>`, `<CPU>`, `<Port>` with the correct values.

What is the value for `<Port>`? 9696

Apply this deployment using the appropriate command and get a list of running Pods. 
You can see one running Pod.

In [42]:
!kubectl apply -f deployment.yaml
!kubectl get deployment
!kubectl get pod

deployment.apps/credit-card created
NAME          READY   UP-TO-DATE   AVAILABLE   AGE
credit-card   0/1     0            0           0s
NAME                           READY   STATUS    RESTARTS   AGE
credit-card-854fcccb98-vdn8l   0/1     Pending   0          0s


## Question 7

Let's create a service for this deployment (`service.yaml`):

```yaml
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <???>
  ports:
  - port: 80
    targetPort: <PORT>
```

Fill it in. What do we need to write instead of `<???>`? credit-card

Apply this config file.

In [43]:
!kubectl apply -f service.yaml
!kubectl get service

service/credit-card-service created
NAME                  TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)        AGE
credit-card-service   LoadBalancer   10.96.156.42   <pending>     80:32056/TCP   3s
kubernetes            ClusterIP      10.96.0.1      <none>        443/TCP        2m49s


## Testing the service

We can test our service locally by forwarding the port 9696 on our computer 
to the port 80 on the service:

```bash
kubectl port-forward service/<Service name> 9696:80
```

Run `q6_test.py` (from the homework 5) once again to verify that everything is working. 
You should get the same result as in Question 1.


In [44]:
!kubectl port-forward service/credit-card-service 9696:80

^C


In [49]:
import requests


url = "http://localhost:9696/predict"

client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
response = requests.post(url, json=client).json()

print(response)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

## Autoscaling

Now we're going to use a [HorizontalPodAutoscaler](https://kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale-walkthrough/) 
(HPA for short) that automatically updates a workload resource (such as our deployment), 
with the aim of automatically scaling the workload to match demand.

Use the following command to create the HPA:

```bash
kubectl autoscale deployment credit-card --name credit-card-hpa --cpu-percent=20 --min=1 --max=3
```

You can check the current status of the new HPA by running:

```bash
kubectl get hpa
```

The output should be similar to the next:

```bash
NAME              REFERENCE                TARGETS   MINPODS   MAXPODS   REPLICAS   AGE
credit-card-hpa   Deployment/credit-card   1%/20%    1         3         1          27s
```

`TARGET` column shows the average CPU consumption across all the Pods controlled by the corresponding deployment.
Current CPU consumption is about 0% as there are no clients sending requests to the server.
> 
>Note: In case the HPA instance doesn't run properly, try to install the latest Metrics Server release 
> from the `components.yaml` manifest:
> ```bash
> kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml
>```

In [48]:
!kubectl autoscale deployment credit-card --name credit-card-hpa --cpu-percent=20 --min=1 --max=3
!kubectl get hpa # on MACOS for live feedback: watch kubectl get hpa

horizontalpodautoscaler.autoscaling/credit-card-hpa autoscaled


Error from server (NotFound): horizontalpodautoscalers.autoscaling "#" not found
Error from server (NotFound): horizontalpodautoscalers.autoscaling "on" not found
Error from server (NotFound): horizontalpodautoscalers.autoscaling "MACOS" not found
Error from server (NotFound): horizontalpodautoscalers.autoscaling "for" not found
Error from server (NotFound): horizontalpodautoscalers.autoscaling "live" not found
Error from server (NotFound): horizontalpodautoscalers.autoscaling "feedback:" not found
Error from server (NotFound): horizontalpodautoscalers.autoscaling "watch" not found
Error from server (NotFound): horizontalpodautoscalers.autoscaling "kubectl" not found
Error from server (NotFound): horizontalpodautoscalers.autoscaling "get" not found
Error from server (NotFound): horizontalpodautoscalers.autoscaling "hpa" not found



## Increase the load

Let's see how the autoscaler reacts to increasing the load. To do this, we can slightly modify the existing
`q6_test.py` script by putting the operator that sends the request to the credit-card service into a loop.

```python
while True:
    sleep(0.1)
    response = requests.post(url, json=client).json()
    print(response)
```

Now you can run this script.

In [18]:
# Load test
from time import sleep
import requests

url = "http://localhost:9696/predict"

client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}

while True:
    sleep(0.01)
    response = requests.post(url, json=client).json()
    #print(response)


KeyboardInterrupt: 

## Question 8 (optional)

Run `kubectl get hpa credit-card-hpa --watch` command to monitor how the autoscaler performs. 
Within a minute or so, you should see the higher CPU load; and then - more replicas. 
What was the maximum amount of the replicas during this test?


* 1
* 2
* 3 [X]
* 4

>Note: It may take a few minutes to stabilize the number of replicas. Since the amount of load is not controlled 
> in any way it may happen that the final number of replicas will differ from initial.